<table align="left" style="border-style: hidden" class="table"> <tr> <td class="col-md-2"><img style="float" src="http://prob140.org/assets/icon256.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Spring 2018</h4><p>Ani Adhikari</div></td></tr></table><!-- not in pdf -->

In [ ]:
from datascience import *
from prob140 import *     
import numpy as np
import pickle

In [ ]:
# Run this code

import itertools as it
from collections import Counter

def clean_string(string):
    """
    Cleans an input string by replacing all newlines with spaces, and then
    removing all letters not in *allowable_letters*
    """
    string = string.replace("\n"," ")
    return "".join([i for i in string.lower().strip() if i in allowable_letters])

def load_bigrams(text):
    """
    Takes a string which has already been cleaned, and returns a dictionary
    of conditional bigram probabilities
    
    cond_bigram[(a,b)] = P(X_{n+1} = b | X_{n} = a)
    
    Uses Laplace smoothing with size $1$, to remove zero transition probabilities
    """
    bigram_counter = Counter(list(it.product(allowable_letters, repeat=2)))
    gram_counter = Counter(allowable_letters*len(allowable_letters))
    for l1, l2 in zip(text,text[1:]):
        bigram_counter[(l1, l2)] += 1
        gram_counter[l1] += 1
    cond_bigram = {k:v/gram_counter[k[0]] for k,v in bigram_counter.items()}
    return cond_bigram
    
def bigram_from_file(filename):
    """
    Given a filename, this reads it, cleans it, and returns the conditional bigram
    """
    file_text = open(filename).read()
    file_text = clean_string(file_text)
    return load_bigrams(file_text)

def reverse_cipher(cipher):
    return {v:k for k, v in cipher.items()}

def print_differences(cipher1, cipher2):
    for k in cipher1:
        if cipher1[k] != cipher2[k]:
            print("%s: %s  %s"%(k,cipher1[k], cipher2[k]))
            
def num_errors(cipher, encoded_text, original_text):
    decoded = np.array(list(decode_text(encoded_text, cipher)))
    original = np.array(list(original_text))
    num_errors = np.count_nonzero(decoded != original)
    return num_errors

def get_secret_text(student_id):
    with open('Lab_06_data/secret_strings.p', 'rb') as f:
        texts = pickle.load(f)
    return texts[student_id % 14]

# Lab 6: Code Breaking by MCMC #

Cryptography is the study of algorithms used to encode and decode messages. Markov Chain Monte Carlo (MCMC) methods have been successfully used to decode messages encrypted using substitution codes and also [more complex](http://probability.ca/jeff/ftpdir/decipherart.pdf) encryption methods. In this lab you will apply MCMC and the Metropolis algorithm to decode English text that has been encrypted by a substitution code.

The lab is based on the paper [The Markov Chain Monte Carlo Revolution](https://math.uchicago.edu/~shmuel/Network-course-readings/MCMCRev.pdf) by [Persi Diaconis](https://en.wikipedia.org/wiki/Persi_Diaconis). It was presented at the 25th anniversary celebrations of MSRI up the hill, and appeared in the Bulletin of the American Mathematical Society in November 2008. The code is based on [Simulation and Solving Substitution Codes](http://www-users.york.ac.uk/~sbc502/decode.pdf) written by [Stephen Connor](https://www.york.ac.uk/maths/staff/stephen-connor/) in 2003.

Recall that in class we worked with a tiny alphabet and a short message that had been encoded using a substitution code. We used a Markov chain model and a scoring system to choose the decoder that had the highest score. That is, we chose the decoder that had the highest likelihood given the message. 

Because the alphabet was small, we were able to list all the decoders and their corresponding scores. When the alphabet is large, making an exhaustive list of all possible decoders and scores is not feasible. That's where the Monte Carlo part of the algorithm comes in. The idea is to choose decoders at random using an algorithm that favors good decoders over bad ones.

In this lab you will work with an alphabet consisting of all 26 English letters as well as a space character that will have a special status. Before you begin, please review Sections [11.3](https://textbook.prob140.org/notebooks-md/11_03_Code_Breaking.html) (Code Breaking) and [11.4](https://textbook.prob140.org/notebooks-md/11_04_Markov_Chain_Monte_Carlo.html) (Markov Chain Monte Carlo) of the textbook. The lab follows those sections closely.

Unlike all the other labs, this lab is not divided into parts. It's just one small project. You will learn how to:
- Apply a decoder to encoded text
- Use transformations for improved numerical accuracy
- Implement the Metropolis algorithm 
- Decode a message encrypted by a substitution code

The programming needed for this lab is more complex than what can reasonably be expected based on Data 8 background, so much of it has been done for you. Those of you who have more extensive Python knowledge might be interested in looking at the details.

Please start by running all the cells above this one.

## Alphabet and Bigrams ##
The text that you are going to decode was written in English. For data on the frequencies of all the different bigrams in English, we will start by counting all the bigrams in *War and Peace* by Tolstoy. That is one of the longest novels in English (actually in Russian, but we are using an English translation from Project Gutenberg) and is often used as a "corpus" or body of language on which to base analyses of other text.

To keep the calculations manageable, we will restrict ourselves to a 27-character alphabet, consisiting of the 26 lower case letters and a space. The cell below places all 27 in a list.

In [ ]:
allowable_letters = list("abcdefghijklmnopqrstuvwxyz ")

In the cell below, we find the relative frequencies of all bigrams in War and Peace and place them in the dictionary `wp_bigrams`. You don't need to know what a dictionary is. It's fine to imagine it containing the same information as a Table that has a column with the bigrams and another column with the relative frequencies. 

In [ ]:
wp_bigrams = bigram_from_file("Lab_06_data/warandpeace.txt")

### 1. Transition Matrix ###
As in class, the model is that the sequence of characters in the text is a Markov chain. The state space of the chain is the alphabet of 27 characters. Of course the text has other punctuation, but we are stripping all of it. We are also replacing upper case letters by lower case.

Construct the transition matrix for this Markov chain based on the relative frequencies in `wp_bigrams` defined above.

We have started the code off for you by defining the transition function `wp_transition`. Use `allowable_letters` in your definition.

In [ ]:
def wp_transition(a, b):
    if (a, b) in wp_bigrams:
        return wp_bigrams[(a, b)]
    return 0

wp_bigram_mc = ...
wp_bigram_mc

## Decoders ##

As you know, when a substitution code has been used for encryption, a decoder is a permutation of the alphabet. We apply this permutation to our encoded text in order to generate the decoded text.

We will be representing the decoder as a Python dictionary. Here is an example of how a dictionary works, using a three-letter alphabet 'a', 'b', 'c'.

If the decoder is ['b', 'c', 'a'] then the substitutions for decoding the encrypted message are

$$
a \to b ~~~~~~~~ b \to c ~~~~~~~~ c \to a
$$

This decoder written as a dictionary dictionary looks like this:

    simple_decoder = {'a':'b','b':'c','c':a'}

To access a value in the decoder, use the method `.get`. For example, to access the translation of *a*, use
    
    simple_decoder.get('a')

In [ ]:
simple_decoder = {'a':'b', 'b':'c', 'c':'a'}
simple_decoder.get('a')

### Keeping Spaces Intact ###
We are assuming that our substitution code keeps spaces unchanged. Letters can precede and follow spaces, but spaces will be fixed points in the encoding permutation. 

So our decoder must keep spaces unchanged as well. The following cell defines a function called `random_decoder` which will randomly generate a decoder. It will operate on the 26 letters, omitting the space.

Run the cell below. The function `random_decoder` constructs a random permutation of the alphabet and places it in a dictionary to make it a decoder.

In [ ]:
decoder_letters = np.array(list("abcdefghijklmnopqrstuvwxyz")) 
# We don't operate on spaces

identity_decoder = \
    {letter:letter for letter in decoder_letters}

# Random starting decoder
def random_decoder():
    """ Random decoder """
    new_letters = decoder_letters.copy()
    np.random.shuffle(new_letters)
    return {orig:new for orig,new in zip(decoder_letters, new_letters)}


#newpage

### 2. Applying a Decoder ###

Define a function `decode_text` that takes as its arguments a string to be decoded and the decoder. It should return the decoded string.

Remember that we are only decoding alphabetical characters. If a character is not in the decoder (e.g. a space), leave the character alone.

In [ ]:

def decode_text(string, decoder):
    new_string = ""
    
    for char in string:
        if char in decoder:
            new_letter = ...
        else:
            new_letter = ...
        
        # Now we append the letter to the back of the new string
        new_string = new_string + new_letter
        
    return new_string

#newpage

## Metropolis Algorithm: Proposal Chain ##
The Metropolis algorithm we will use is essentially the same as the one defined in class and proved to converge to the correct distribution. There is one tweak to increase computatational accuracy, and another to deal with spaces. You shouldn't worry about either of them.

Please refer to Section [11.4](https://textbook.prob140.org/notebooks-md/11_04_Markov_Chain_Monte_Carlo.html) of the textbook for the notation being used below.

- The state space $S$ of the proposal chain is the set of all 26! possible decoders, that is, all 26! possible permutations of letters.

- The proposal transition matrix $Q$ is chosen as follows. Given that the chain is at decoder $i$, pick another decoder by randomly swapping two elements of $i$. Thus for any two decoders $i$ and $j$,

\begin{equation}
Q(i, j) = 
 \begin{cases}
 \frac{1}{\binom{26}{2}} & \text{if } i \text{ and } j \text{ differ in exactly two places} \\
0 & \text{otherwise}
 \end{cases}
\end{equation}

Notice that $Q$ is symmetric, as required by the algorithm.

**Note:** This proposal chain is not aperiodic. But it doesn't have to be. As discussed in class, the process in the algorithm uses the proposal chain as well as randomization, and is both irreducible and aperiodic by construction.

### 3a) Irreducibility ###
The state space of the proposal chain is large. Run the cell below to see how large.

In [ ]:
np.math.factorial(26)

The proposal chain has to be irreducible and have a symmetric transition matrix. We have shown above that $Q$ is symmetric. Explain why the proposal chain is irreducible.

**Your answer here.**

### 3b) Making a Proposal to Move ###

Define the function `generate_proposed_decoder` which takes an existing decoder and returns a new decoder as follows: it randomly picks two different letters of the alphabet and swaps the decoding for those letters.

For example, if we have the decoder `{'a':'b','b':'c','c':a'}` and decide to swap `a` and `b`, our new decoder should be `{'a':'c','b':'b','c':a'}`. 

There are several different ways you can use Python to draw random samples. In this instance we recommend `np.random.choice`. It takes two arguments:
- a list from which to sample uniformly at random with replacement
- the number of times to sample

The optional argument `replace=False` leads to simple random sampling.

It returns an array consisting of the sampled elements.

In [ ]:
def generate_proposed_decoder(decoder):
    new_decoder = decoder.copy()
    
    
    letter1 = ...
    letter2 = ...
    
    new_value_of_letter1 = ...
    new_value_of_letter2 = ...

    # This code replaces the value of letter1 and letter2 
    # with new_value_of_letter1 and new_value_of_letter2
    new_decoder[letter1] = new_value_of_letter1
    new_decoder[letter2] = new_value_of_letter2
    return new_decoder


## The Target Distribution $\pi$ ##

For letters $x_1$ and $x_2$, let $B(x_1, x_2)$ be the $(x_1, x_2)$th entry in the bigram transition matrix. Suppose the decoded text consists of the characters $x_0, x_1, \ldots , x_n$. Recall from class that we defined the *score* of the corresponding decoder $i$ to be

$$
s(i) = \prod_{k=0}^{n-1}B_i(x_k, x_{k+1})
$$

We would like Decoder $i$ to appear among our randomly selected decoders with target probability $\pi(i)$, where $\pi(i)$ is the likelihood of $i$ given the data. As we saw in class, the likelihood $\pi(i)$ is proportional to $s(i)$ but we don't know the constant of proportionality. However, for any two decoders $i$ and $j$, we can calculate the ratio

$$
\frac{\pi(j)}{\pi(i)} = \frac{s(j)}{s(i)}
$$

Here's a tweak to what was done in class. When the string is long, the score of any decoder is going to be very small. This causes problems with numerical accuracy. So we will work with the log-score instead, that is, with the $\log$ of the score. 

$$
\log(s(i)) = \sum_{k=0}^{n-1} \log(B_i(x_k, x_{k+1}))
$$


#newpage

### 4. Log Probability of Path ###

Define a function `log_prob_path` that takes two arguments:
- a string `message` that is a sequence of letters 
- the bigram transition matrix of a `MarkovChain` `mc` 

and returns the log of the probability of the path `message` taken by `mc`, given that the first character of `message` is the initial state of the path.

Though `mc.prob_of_path` returns the probability of the path, don't use `np.log(mc.prob_of_path)`. Each path has very small probability, so the numbers will round too soon and your computations might be inaccurate.

Instead, you can use the function [`mc.log_prob_of_path`](https://probability.gitlab.io/prob140/_autosummary/prob140.MarkovChain.log_prob_of_path.html#prob140.MarkovChain.log_prob_of_path). This takes as its first argument the starting character of the string, and as its next argument a list or array containing the characters in the rest of the string. It returns the log of the probability of the path, conditional on the starting character.

Some string methods will be helpful:

* string[0] returns the first letter of a string. For example:
```python
>>> x = 'HELLO'
>>> x[0]
'H'
```
* string[1:] returns everything except the first letter of a string. For example:
```python
>>> x[1:]
'ELLO'
```
* list(string) splits the string into a list of its characters. This was used above to generate `allowable_letters`.

In [ ]:
def log_prob_path(string, bigram_mc):
    start = ...
    
    path = list(...)
    
    return bigram_mc.log_prob_of_path(start, path)

#newpage

### 5. Log Score of Decoder ###

Define a function `log_score` that takes the following arguments:

- an encrypted string
- a decoder
- a bigram transition matrix

The function should decode the encrypted string using the decoder, and return the log score of the decoder.

Use the functions `decode_text` and `log_prob_path` that you wrote earlier.

In [ ]:
def log_score(string, decoder, bigrams):
    return ...

### Acceptance Ratios ###
The Metropolis algorithm creates a new chain with $\pi$ as its limit distribution. Recall that if the new chain is at $i$, then the the decision about whether or not to accept a transition to a new state $j$ depends on the ratio $r(i, j)$ defined by

$$
r(i, j) ~ = ~ \frac{\pi(j)}{\pi(i)} ~ = ~ \frac{s(j)}{s(i)}
$$

Because we are working with log scores and not the scores themselves, every statement that we made in class about scores has to be rewritten in terms of log scores. Observe that

$$
\frac{s(j)}{s(i)} = e^{\log(\frac{s(j)}{s(i)})} = e^{\log(s(j)) - \log(s(i))}
$$

Since the algorithm involves comparing $r(i, j)$ to 1, note that

$$
\frac{s(j)}{s(i)} < 1 ~~~ \iff ~~~ s(j) < s(i) ~~~ \iff ~~~
\log(s(j)) < \log(s(i)) 
$$

because $\log$ is an increasing function.

#newpage

## Implementing the Metropolis Algorithm ##

### 6. An Encoded Message ###

Enter your student id in the cell below to access your secret encoded message. By the end of the lab, you should be able to decode your string.

In [ ]:
student_id = ...
secret_text = get_secret_text(student_id)
secret_text

#newpage

### 7. Implementation ###

Define the function `metropolis` which will take as its arguments:

- the encrypted text, as a string
- the transition matrix of bigrams
- the number of repetitions for running the Metropolis algorithm 

The function should return the decoder that it arrives at after performing the Metropolis algorithm the specified number of times. 

If the number of repetitions is large, you know from class that the distribution of this random decoder is close to the desired distribution $\pi$ which is the likelihood distribution of the decoder given the data. Therefore you expect to end up with a decoder that has a high likelihood compared to the other decoders.

Between iterations, you should keep track of the following variables:

1. `bestDecoder`: the decoder that has the highest log score
2. `bestScore`: the log score associated with the bestDecoder
3. `decoder`: the decoder that you are currently working with
4. `lastScore`: the log score of the current decoder

In each iteration, you should do the following:

1. Generate a new decoder based on the current decoder; the "new" decoder might be the same as the current one.
2. Calculate the log score of your new decoder.
3. **Important:** Follow the Metropolis algorithm to decide whether or not to move to a new decoder.
4. If the new decoder's log score is the best seen so far, update `bestScore` and `bestDecoder`

In [ ]:
import time

def p_coin(p):
    """
    Flips a coin that comes up heads with probability p
    and returns 1 if Heads, 0 if Tails
    """
    return np.random.random() < p


def metropolis(string_to_decode, bigrams, reps):
    decoder = random_decoder() # Starting decoder
    
    bestDecoder = decoder
    bestScore = log_score(string_to_decode, bestDecoder, bigrams)
    
    lastScore = bestScore
    
    for rep in np.arange(reps):
        
        # This will print out our progress
        if rep*10%reps == 0: # Repeat every 10%
            time.sleep(.01)
            decoded_text = decode_text(\
                           string_to_decode, bestDecoder)[:40]
            print('Score: %.00f \t Guess: %s'%(bestScore, decoded_text))
            
        #########################
        # Your code starts here #
        #########################
        proposed_decoder = ...
        log_s_orig = ...
        log_s_new = ...
        
        if log_s_new > log_s_orig or p_coin(np.e**(log_s_new - log_s_orig)): 
            # If better than before or p-coin flip works
            
            ...
            
            if log_s_new > bestScore:
                ...
    return bestDecoder

#newpage

## Running It ##

The text you are going to first encrypt and then decode is taken from [Winston Churchill's speech](https://en.wikipedia.org/wiki/We_shall_fight_on_the_beaches) to the House of Commons on 4 June 1940. Naturally, it contains upper case letters and punctuation that we have not included in our alphabet. In the cell below, the function `clean_string` strips out all the punctuation and replaces upper case letters by the corresponding lower case letters.

In [ ]:
ogtext = """
I have, myself, full confidence that if all do their duty, if nothing is neglected, 
and if the best arrangements are made, as they are being made, we shall prove ourselves 
once again able to defend our Island home, to ride out the storm of war, and to outlive 
the menace of tyranny, if necessary for years, if necessary alone. At any rate, that is 
what we are going to try to do. That is the resolve of His Majesty’s Government-every man 
of them. That is the will of Parliament and the nation. 
"""
original_string = clean_string(ogtext)
encoding_cipher = random_decoder()
string_to_decode = decode_text(original_string,encoding_cipher)
string_to_decode


### 8a) Finding the Decoder ###
Run the following cell to generate your decoder. You may have to run the cell a couple of times to get the correct decoder.

In [ ]:
new_decoder = metropolis(string_to_decode, wp_bigram_mc, 10000) 

Run the cell below to decode the entire string.

In [ ]:
decode_text(string_to_decode, new_decoder)

### 8b) Decoding Your Secret Message

`secret_text` contains your customized, secret message. Use MCMC to decode it.

In [ ]:
secret_decoder = metropolis(...)
decode_text(secret_text, secret_decoder)

## Conclusion ##
What you have learned:
- How to implement the Metropolis algorithm to decode text encrypted by a substitution code, with simplifying assumptions about spaces, capitalization, and punctuation
- MCMC works
- Abstract concepts such as balance and convergence of $n$-step transition matrices are remarkable in their own right and also have powerful applications

In [ ]:
import gsExport
gsExport.generateSubmission("Lab_05.ipynb")